In [1]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 41.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.


In [2]:
!pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/bi_influence.git

  Cloning https://github.com/Shannu3766/bi_influence.git to /tmp/pip-req-build-h56y1wgp
  Running command git clone --filter=blob:none --quiet https://github.com/Shannu3766/bi_influence.git /tmp/pip-req-build-h56y1wgp
  Resolved https://github.com/Shannu3766/bi_influence.git to commit 981f2a3a9b906864b4f65765cb42335892489a46
  Preparing metadata (setup.py) ... done
  Created wheel for adaptive_lora: filename=adaptive_lora-2.1.0-py3-none-any.whl size=5843 sha256=01d3241a2368f136d7801ed8ef2368f40abe7f90177401a6c24c253f9b933de8
  Stored in directory: /tmp/pip-ephem-wheel-cache-1745quu8/wheels/4b/91/1d/e153ff3aa3759d6ccdcc4706d8559f1cbfaa4aa22b992958cd
Successfully built adaptive_lora


In [3]:
import warnings
import os
import logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
# logging.set_verbosity_error()

In [4]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from torch.utils.data import DataLoader
from adaptive_lora.callbacks import AdaptiveLoRACallback
import numpy as np
print("torch:", torch.__version__)
print("cuda devices:", torch.cuda.device_count())

2025-12-03 08:05:31.496431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764749131.667909      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764749131.725405      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

torch: 2.6.0+cu124
cuda devices: 2


In [5]:
import os
import warnings
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    logging
)
from peft import get_peft_model, LoraConfig
import evaluate

In [6]:
# Silence all unnecessary outputs
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
logging.set_verbosity_error()


In [ ]:
from datasets import load_dataset
import time

def load_dataset_with_retry(path, name=None, max_retries=None, wait=2):

    attempt = 0

    while True:
        try:
            if name is not None:
                ds = load_dataset(path, name)
            else:
                ds = load_dataset(path)

            print(f"Dataset loaded successfully after {attempt} attempts.")
            return ds

        except Exception as e:
            attempt += 1
            wait_time = wait * min(5, attempt)  

            print(f"[Attempt {attempt}] Failed to load dataset: {e}")
            print(f"Retrying in {wait_time} seconds...\n")

            if max_retries is not None and attempt >= max_retries:
                print("Max retries reached. Raising error.")
                raise e

            time.sleep(wait_time)
dataset = load_dataset_with_retry("glue", "sst2")


[Attempt 1] Failed to load dataset: 429 Client Error: Too Many Requests for url: https://us.gcp.cdn.hf.co/xet-bridge-us/621ffdd236468d709f181e3f/62e400c3e48b67de004b9587b6115032fbd86d306448eb4d547087b05b59a60e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train-00000-of-00001.parquet%3B+filename%3D%22train-00000-of-00001.parquet%22%3B&Expires=1764752756&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY0NzUyNzU2fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjIxZmZkZDIzNjQ2OGQ3MDlmMTgxZTNmLzYyZTQwMGMzZTQ4YjY3ZGUwMDRiOTU4N2I2MTE1MDMyZmJkODZkMzA2NDQ4ZWI0ZDU0NzA4N2IwNWI1OWE2MGVcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=q4luxfW1K6ggBR27P3ywLv~PpBdxglTJR2uRR2V4WeoundSEkq0F8wc--gskNTdcFkJL5iCyqRmM1FviNnleLZ2Xh0gL6hZP4BYRhs8FC2Z3jMbaTPBV1y7JVNAFfDdREUZZyPS-wiJ33k9FY-KdnidtTW067fs-eLygjvsZIeHTIea4OLdoRdGHf8RP07mpTVoeF68aLZmYuScpsIgLecdMvhrOkM4jHn1gmOhsii3iW2NRx2d~9rtf-gTtxxrK7lZk6CB4zRR3K3zGM1vU45SRIw

[Attempt 2] Failed to load dataset: 429 Client Error: Too Many Requests for url: https://us.gcp.cdn.hf.co/xet-bridge-us/621ffdd236468d709f181e3f/22fc04a15caeb9f814ca92f0e4a3e04646e72e2e1b0587ef8d82cc8d1e61fe79?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27validation-00000-of-00001.parquet%3B+filename%3D%22validation-00000-of-00001.parquet%22%3B&Expires=1764752760&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY0NzUyNzYwfX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjIxZmZkZDIzNjQ2OGQ3MDlmMTgxZTNmLzIyZmMwNGExNWNhZWI5ZjgxNGNhOTJmMGU0YTNlMDQ2NDZlNzJlMmUxYjA1ODdlZjhkODJjYzhkMWU2MWZlNzlcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=D-yEdEIP0KE4-LnpZR3YRzYqIaXm~DSwks8yc6Vn1xxgupwfT8aHdJrQDdjjoEMAkwLMfj7u9ysYVKkQHcPhZUFCtD7ByGhSf0Ig6flGagiFAx5aYxp3NdI2AIWaXvjEUHxAnSJrQ9i2UZzsES1p4Ih8EOz46gI-gpoQ-~dXYzoNc3SaiquMwaRC3AaCcMzCPs5LiLjDBLCSTSgqz0190ud6JjaYxGQ-56mgNVOCbWrcaSJuZPxBCafshvGMpJJCYlaeQqS4RykhKLVm

[Attempt 3] Failed to load dataset: 429 Client Error: Too Many Requests for url: https://us.gcp.cdn.hf.co/xet-bridge-us/621ffdd236468d709f181e3f/22fc04a15caeb9f814ca92f0e4a3e04646e72e2e1b0587ef8d82cc8d1e61fe79?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27validation-00000-of-00001.parquet%3B+filename%3D%22validation-00000-of-00001.parquet%22%3B&Expires=1764752765&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY0NzUyNzY1fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjIxZmZkZDIzNjQ2OGQ3MDlmMTgxZTNmLzIyZmMwNGExNWNhZWI5ZjgxNGNhOTJmMGU0YTNlMDQ2NDZlNzJlMmUxYjA1ODdlZjhkODJjYzhkMWU2MWZlNzlcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=lsv0guI6fevaUcRPih~laNnadIOfpVlaM4xviJi9AISwdjqJ4q~knntwuvoyfP9WVmeb-1HdVwz7ycchzTlzciR-bDKfTHuBwD2RLPQIizd91P4sKILwGwbf14KV7~wJjpFqaesdvk5VWDHIddSzibNq5147M5bsn0BB4DHTFi6d16mYU~XVY-XSaupdSpMcFU9br5iR8kAfizTYLNBeIvmjGnbQ-TpTYOxjBMmWpU6zK1zKrV~~iGlCLStKFK7jNV0UHwOA5raxAFz7

[Attempt 4] Failed to load dataset: 429 Client Error: Too Many Requests for url: https://us.gcp.cdn.hf.co/xet-bridge-us/621ffdd236468d709f181e3f/22fc04a15caeb9f814ca92f0e4a3e04646e72e2e1b0587ef8d82cc8d1e61fe79?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27validation-00000-of-00001.parquet%3B+filename%3D%22validation-00000-of-00001.parquet%22%3B&Expires=1764752773&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY0NzUyNzczfX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjIxZmZkZDIzNjQ2OGQ3MDlmMTgxZTNmLzIyZmMwNGExNWNhZWI5ZjgxNGNhOTJmMGU0YTNlMDQ2NDZlNzJlMmUxYjA1ODdlZjhkODJjYzhkMWU2MWZlNzlcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=R0xxunSjmBOQMcxjrahR~KgvUENnn49aZuN1sca2Dmlt0P340GWxexSXL1a8TTFx9HjPSFTLpAXZk419hLhqRnqasTFeULJ6ROpftkpNH3R7uqVs3~uhn5RZq9EG~KR76JZm5YDgCM-Sl0aXHxegOW~-6TiysBdiYIN7bfeQpGmoXNBp28EC1XeqRP0X9lQck6JhleoQD1A7bmBne2mGHO9vmYI4qTw0CYTtgaCVxPMPT9~LZBp7jlR2U9D9f2xSASS6BzkUAPrigJT9

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset loaded successfully after 4 attempts.


In [ ]:
model_checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    device_map="auto"
)

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

MAX_LEN = 128  


def clean_text(s):
    if s is None:
        return ""
    return " ".join(str(s).strip().split())


def tokenize_function(examples):

    texts = [
        f"Review: {clean_text(x)}\nSentiment:"
        for x in examples["sentence"]
    ]

    return tokenizer(
        texts,
        truncation=True,
        padding=False,   
        max_length=MAX_LEN,
    )


tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["sentence", "idx"],
)

if "label" in tokenized_dataset["train"].column_names:
    tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

tokenized_dataset.set_format("torch")


data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    pad_to_multiple_of=8,  
)


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
accuracy = evaluate.load("accuracy")
rank = 8
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return {"accuracy": accuracy.compute(predictions=preds, references=labels)}

peft_config = LoraConfig(
    task_type="SEQ_CLS",
    r=rank,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[   
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ]
)
model = get_peft_model(model, peft_config)


In [ ]:
from peft.tuners.lora import LoraLayer

def get_lora_module_names(peft_model):
    names = []
    for name, module in peft_model.named_modules():
        if isinstance(module, LoraLayer):
            names.append(name)
    return names

lora_names = get_lora_module_names(model)   
print(f"🔢 Number of LoRA modules: {len(lora_names)}\n")

num_lora_modules = len(lora_names)
TOTAL_RANK_BUDGET = num_lora_modules*rank  

avg_rank_per_module = TOTAL_RANK_BUDGET / num_lora_modules
print(
    f"💡 With TOTAL_RANK_BUDGET={TOTAL_RANK_BUDGET} over "
    f"{num_lora_modules} modules, avg rank ≈ {avg_rank_per_module:.2f}"
)


🔢 Number of LoRA modules: 154

💡 With TOTAL_RANK_BUDGET=1232 over 154 modules, avg rank ≈ 8.00


In [12]:
training_args = TrainingArguments(
    disable_tqdm=False,
    output_dir="./SST2_adaptive_results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    logging_steps=500,
    eval_strategy="epoch",
    eval_steps=500,
    bf16=True,              
    report_to="none",
    save_strategy="no",
)


In [13]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [14]:

from torch.utils.data import DataLoader
train_dataset = tokenized_dataset["train"].select(range(10000))
eval_dataset = tokenized_dataset["validation"]

val_dataloader = DataLoader(
    eval_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=data_collator
)

In [15]:
from adaptive_lora.callbacks import AdaptiveLoRACallback

adaptive_callback = AdaptiveLoRACallback(
    val_dataloader=val_dataloader,
    total_rank=TOTAL_RANK_BUDGET,
    tau=0.9,      
    verbose=True
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[adaptive_callback],
)

In [17]:
trainer.train()


--- AdaptiveLoRA: Preparing ranks for Epoch 1 ---
Computing BI importance scores (pre-training)...


Computing BI scores:   0%|          | 0/109 [00:00<?, ?it/s]

Allocating new ranks based on BI scores...
Applying new ranks to LoRA modules for this epoch...
  - base_model.model.model.layers.0.self_attn.q_proj: r=8 (Unchanged, Score: 0.9897)
  - base_model.model.model.layers.0.self_attn.k_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.self_attn.v_proj: r=8 (Unchanged, Score: 0.9963)
  - base_model.model.model.layers.0.self_attn.o_proj: r=8 (Unchanged, Score: 0.9941)
  - base_model.model.model.layers.0.mlp.gate_proj: r=8 (Unchanged, Score: 0.9943)
  - base_model.model.model.layers.0.mlp.up_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.mlp.down_proj: r=8 (Unchanged, Score: 0.9992)
  - base_model.model.model.layers.1.self_attn.q_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.k_proj: r=8 (Unchanged, Score: 0.9921)
  - base_model.model.model.layers.1.self_attn.v_proj: r=8 (Unchanged, Score: 0.9870)
  - base_model.model.model.layers.1.self_attn.o_proj: r=8 → 7 (Sc

Epoch,Training Loss,Validation Loss,Accuracy
1,0.217700,0.242424,{'accuracy': 0.9438073394495413}
2,0.167700,0.276584,{'accuracy': 0.944954128440367}
3,0.141100,0.371025,{'accuracy': 0.9403669724770642}


📄 Epoch 1: Rank allocations logged to ./logs/adaptive_lora_epoch_logs.csv


--- AdaptiveLoRA: Preparing ranks for Epoch 2 ---
Computing BI importance scores (pre-training)...


Computing BI scores:   0%|          | 0/109 [00:00<?, ?it/s]

Allocating new ranks based on BI scores...
Applying new ranks to LoRA modules for this epoch...
  - base_model.model.model.layers.0.self_attn.q_proj: r=8 (Unchanged, Score: 0.9897)
  - base_model.model.model.layers.0.self_attn.k_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.self_attn.v_proj: r=8 (Unchanged, Score: 0.9973)
  - base_model.model.model.layers.0.self_attn.o_proj: r=8 (Unchanged, Score: 0.9883)
  - base_model.model.model.layers.0.mlp.gate_proj: r=8 (Unchanged, Score: 0.9913)
  - base_model.model.model.layers.0.mlp.up_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.mlp.down_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.q_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.k_proj: r=8 (Unchanged, Score: 0.9912)
  - base_model.model.model.layers.1.self_attn.v_proj: r=8 → 7 (Score: 0.9854)
  - base_model.model.model.layers.1.self_attn.o_proj: r=7 (Unchanged, Sc

Computing BI scores:   0%|          | 0/109 [00:00<?, ?it/s]

Allocating new ranks based on BI scores...
Applying new ranks to LoRA modules for this epoch...
  - base_model.model.model.layers.0.self_attn.q_proj: r=8 (Unchanged, Score: 0.9897)
  - base_model.model.model.layers.0.self_attn.k_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.self_attn.v_proj: r=8 (Unchanged, Score: 0.9977)
  - base_model.model.model.layers.0.self_attn.o_proj: r=8 (Unchanged, Score: 0.9876)
  - base_model.model.model.layers.0.mlp.gate_proj: r=8 (Unchanged, Score: 0.9907)
  - base_model.model.model.layers.0.mlp.up_proj: r=8 (Unchanged, Score: 0.9994)
  - base_model.model.model.layers.0.mlp.down_proj: r=8 (Unchanged, Score: 0.9992)
  - base_model.model.model.layers.1.self_attn.q_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.k_proj: r=8 (Unchanged, Score: 0.9900)
  - base_model.model.model.layers.1.self_attn.v_proj: r=7 (Unchanged, Score: 0.9859)
  - base_model.model.model.layers.1.self_attn.o_proj: r=7 (Unchan

TrainOutput(global_step=1875, training_loss=0.1586923828125, metrics={'train_runtime': 3626.2366, 'train_samples_per_second': 8.273, 'train_steps_per_second': 0.517, 'total_flos': 8733387507695616.0, 'train_loss': 0.1586923828125, 'epoch': 3.0})

In [ ]:
met = trainer.evaluate()
print(met)

{'eval_loss': 0.3013925552368164, 'eval_accuracy': {'accuracy': 0.9492349174311926}, 'eval_runtime': 49.7935, 'eval_samples_per_second': 17.512, 'eval_steps_per_second': 2.189, 'epoch': 3.0}
